# 0 Imports

In [2]:
import pickle
import datetime

import numpy as np
import pandas as pd
import seaborn as sns


from matplotlib import pyplot
from prophet    import Prophet

from IPython.display        import Image
from IPython.core.display   import HTML




In [ ]:
# Supressão da notação científica.
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.6f}'.format)

## 0.1 Funções Suporte

In [ ]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [18,8]
    plt.rcParams['font.size'] = 14
    
    display( HTML('<style>.container { width: 100% !important;} </style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False)
    
    sns.set()
    
jupyter_settings()

In [ ]:
def abrir_arquivo()-> pd.DataFrame:
    with open(f"{__CAMINHO_INTERIM}{__NOME_ARQUIVO_IMPORTAR}","rb") as arquivo:
        df = pickle.load(arquivo)
    return df

In [ ]:
def exportar_dados_preparados(df: pd.DataFrame):
    pickle.dump(f"{__CAMINHO_INTERIM}{__NOME_ARQUIVO_EXPORTAR}","rb")

## 0.2 Load Data

In [ ]:
__CAMINHO_RAW = '../data/raw/'
__CAMINHO_INTERIM = '../data/interim/'

__NOME_ARQUIVO_IMPORTAR = '3_0_feature_engineering.pkl'
__NOME_ARQUIVO_EXPORTAR = '5_0_preparacao_dados.pkl'

In [ ]:
df = abrir_arquivo()

# 5.0 Preparação dos Dados

Observações do ponto 4.0 EDA pós FE:

### Variáveis Numéricas

Normalização:
1. Principalmente para a própria variável resposta de custo, tem muito outlier.

Reescala nos dados numéricos, testar e verificar reescala robusta a outliers:
1. Os custos derivativos (Agregados), até que estão bonitinhos, talvez não dê muito trabalho.
2. As datas construídas, talvez precisem de uma transformação, pois são cíclicas
3. As datas não cíclicas (ano_semana, ano_mes), vê como faz, qual transformação aplicar, ou só descartar, elas são 'str'.

Transformação:
1. A coluna date passar para sen e cos (cíclico).
2. Posso até fazer isso para todas as datas (anos, mês, dia, dia_da_semana, semana_ano)(cíclico).

### Variáveis Categóricas

## 5.1 Normalização

## 5.2 Reescala

## 5.3 Transformação

# 5.9 Export PKL

In [ ]:
exportar_dados_preparados(df=df)